In [81]:
from tool.preprocess import Preprocess
from tool.s3_connect import S3_connector
from textrank import run_textrank
import pandas as pd
from wordToVector import WordToVector
from tqdm import tqdm
from database.db import *
from tqdm import tqdm
from scipy.spatial import distance
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np

In [115]:
s3_connector = S3_connector()
run = run_textrank.RunTextRank()
keys = run.key_extractor()
w = WordToVector()

In [116]:
data = select_not_null('정치', '2020-11-02')

In [83]:
data = get_current_articles('정치',300)

In [117]:
len(data)

1165

In [118]:
# headline = []
# for i in tqdm(ids):
#     headline.append(get_header(i[1]))


In [119]:
data[0]

('100-023-0003573085',
 '{"다른 민주당 관계자는 “당헌과 함께 수정할 부칙 조항에 이번 전 당원투표 등을 거쳐 결정한 서울·부산시장 보궐선거 공천 부분은 추가로 전 당원투표를 할 필요가 없다는 내용을 넣을 예정 이라고 했다","민주당의 한 의원은 “서울·부산시장 선거에 공천할 수밖에 없는 현실적 이유 때문에 의결 권한도 없는 전 당원투표를 명분쌓기 목적으로 활용한 것 이라며 “당헌을 무시했다는 비판은 피하기 어려워 보인다”고 했다.","더불어민주당의 내년 4월 서울·부산시장 보궐선거 공천 및 당헌 개정 여부를 묻는 전당원투표 안내문"}',
 '{투표,당원,민주당,당헌,공천,서울,부산,시장,선거}',
 datetime.date(2020, 11, 2))

In [120]:
vectors = []
key_sentences = []
keywords = []
indexs = []
for idx, key_sentence, keyword, date in data:
    if type(w.vectorize_word_list(keyword[1:-1].split(","))) is float:
        continue
    indexs.append(idx)
    key_sentences.append(key_sentence[1:-1].split(","))
    keywords.append(keyword[1:-1].split(","))
    w_vector = (w.vectorize_word_list(keyword[1:-1].split(","))).tolist()
    vectors.append(w_vector)


In [121]:
# headline = list(map(lambda x : [x[1]], headline))

In [122]:
df = pd.DataFrame({'news_id':indexs,'key_sentence':key_sentences,'keywords':keywords,'vectors':vectors})
df.head()

,news_id,key_sentence,keywords,vectors
0,100-023-0003573085,"[""다른 민주당 관계자는 “당헌과 함께 수정할 부칙 조항에 이번 전 당원투표 등을 ...","[투표, 당원, 민주당, 당헌, 공천, 서울, 부산, 시장, 선거]","[-0.15592455863952637, -0.014079248532652855, ..."
1,100-421-0004964731,"[""검찰 역사상 가장 수치스러운 사건…스스로 돌이키지 않으면 국회가 나설 것 김종민...","[검찰, 위원, 최고, 조사, 차관, 대통령, 국회, 수사]","[0.7033575773239136, -0.09183067083358765, 1.2..."
2,100-009-0004686516,"[""재산세를 완화할 중저가 1주택 기준으로 정부와 청와대는 공시가 6억원 이하를 민...","[재산세, 이하, 정부, 완화, 기준, 민주당, 청와대, 주택, 대주주]","[0.24903562664985657, 0.10665787756443024, 1.1..."
3,100-629-0000049815,"[""더불어민주당은 내년 4월 서울·부산시장 보궐선거 공천을 위한 당헌 개정 전당원 ...","[당헌, 개정, 선거, 보궐, 민주당, 시장, 당원, 서울, 부산]","[-0.2024742215871811, -0.04975862428545952, 0...."
4,100-082-0001039759,"[""리얼미터 조사 기준으로 이낙연 대표는 6개월 연속 하락하며 처음으로 단독 1위를...","[대표, 지사, 이재명, 이낙연, 조사, 의원, 구도, 리얼미터, 총장]","[0.19289469718933105, 0.29414889216423035, 0.3..."


In [89]:
def map_id_index(df):
    map_info = {}
    for i in range(df.shape[0]):
        map_info[df.index[i]] = df["news_id"][i]
    return map_info
print(map_id_index(df))

{0: '100-023-0003577967', 1: '100-008-0004503191', 2: '100-011-0003829957', 3: '100-020-0003321868', 4: '100-032-0003044453', 5: '100-032-0003044450', 6: '100-032-0003044444', 7: '100-052-0001516967', 8: '100-001-0012030937', 9: '100-022-0003525533', 10: '100-011-0003829609', 11: '100-052-0001516958', 12: '100-052-0001516955', 13: '100-052-0001516953', 14: '100-052-0001516946', 15: '100-052-0001516945', 16: '100-001-0012030915', 17: '100-003-0010198822', 18: '100-056-0010938592', 19: '100-119-0002445360', 20: '100-056-0010938561', 21: '100-056-0010938560', 22: '100-025-0003054304', 23: '100-055-0000856162', 24: '100-022-0003525531', 25: '100-055-0000856159', 26: '100-055-0000856158', 27: '100-214-0001081088', 28: '100-057-0001521023', 29: '100-057-0001521017', 30: '100-421-0005004326', 31: '100-057-0001521011', 32: '100-057-0001521013', 33: '100-022-0003525530', 34: '100-023-0003577886', 35: '100-449-0000200674', 36: '100-448-0000311811', 37: '100-449-0000200672', 38: '100-449-00002006

In [90]:
## cosine similarity ###
base = 1
def neighbor(df, base, pivot=0.9):
    
    neighbors = []
    cosine = {}
    
    for i in range(len(df)):
        cosine[i] = 1 - distance.cosine(df['vectors'][base], df['vectors'][i])
        
    cosine_new = list(cosine.items())
    cosine_new.sort(key=lambda x:x[1], reverse=True)
    
    for i, j in cosine_new:
        if j > pivot and i != base:
            neighbors.append(i)

    return neighbors

In [91]:
def all_neighbor(df, base, pivot=0.9):
    stack = [base]
    searched = {}
    
    while len(stack) != 0:
        target = stack.pop()
        neighbors = neighbor(df, target, pivot)        
        
        for n in neighbors:
            if not searched.get(n, False):
                stack.append(n)

        searched[target] = True
    
    searched = list(searched.keys())
    return searched

In [92]:
def clustering(df, pivot=0.9):
    unsearched = dict(zip(range(df.shape[0]), [False]*(df.shape[0])))
    cluster = []
    
    while len(unsearched.keys()) != 0:
        target = unsearched.popitem()[0]
        neighbors = all_neighbor(df, target, pivot)
        
        print(target, neighbors)
        
        for n in neighbors:
            if target != n:
                unsearched.pop(n)
        
        cluster.append(neighbors)

    return cluster

In [93]:
cl = clustering(df)

297 [297]
296 [296, 170, 96, 148, 150, 183, 174, 98, 84, 173, 282, 0, 147, 155, 196, 115, 33, 161, 182, 50, 158, 252, 97, 284, 91, 127, 80]
295 [295, 19, 38, 285, 231, 111, 255, 123, 162, 41, 78, 71, 1, 72, 77, 249, 242, 179, 269, 109, 30, 35, 270, 244, 276, 117, 62, 247, 200, 12, 67]
294 [294]
293 [293, 129, 134, 125, 107]
292 [292, 227]
291 [291]
290 [290]
289 [289]
288 [288]
287 [287, 219, 20]
286 [286, 181]
283 [283]
281 [281]
280 [280, 103, 16]
279 [279]
278 [278]
277 [277, 79, 175, 131, 268, 143, 108, 140, 132, 137, 26, 160]
275 [275]
274 [274]
273 [273]
272 [272, 114]
271 [271, 176, 265, 121]
267 [267]
266 [266]
264 [264]
263 [263]
262 [262]
261 [261]
260 [260]
259 [259]
258 [258, 145]
257 [257]
256 [256]
254 [254]
253 [253]
251 [251, 163, 139, 113]
250 [250]
248 [248]
246 [246]
245 [245]
243 [243]
241 [241]
240 [240]
239 [239]
238 [238]
237 [237]
236 [236]
235 [235]
234 [234]
233 [233]
232 [232]
230 [230]
229 [229]
228 [228]
226 [226, 223]
225 [225]
224 [224]
222 [222, 69, 82, 

In [42]:
map1 = map_id_index(df)

In [48]:
for i in range(len(cl[1])):
    cl[1][i] = map1[cl[1][i]]
cl[1]

['100-005-0001383153',
 '100-023-0003577861',
 '100-014-0004532810',
 '100-123-0002234703',
 '100-022-0003525486',
 '100-277-0004795975',
 '100-023-0003577857',
 '100-025-0003054271',
 '100-052-0001516862',
 '100-005-0001383180',
 '100-014-0004532720',
 '100-023-0003577967',
 '100-011-0003829560',
 '100-015-0004453689',
 '100-417-0000621840',
 '100-082-0001044852',
 '100-022-0003525530',
 '100-421-0005003939',
 '100-052-0001516799',
 '100-022-0003525524',
 '100-629-0000053541',
 '100-022-0003525459',
 '100-119-0002445338',
 '100-008-0004502894',
 '100-003-0010198588',
 '100-052-0001516836',
 '100-023-0003577873']

In [79]:
for i in cl[1]:
    print((df[df["news_id"] == i].keywords))

296    [주택, 임대, 아파트, 서울, 위원장, 매입, 강조]
Name: keywords, dtype: object
170    [아파트, 의원, 서울, 주택, 민주당, 이낙연, 임대, 빌라, 대표]
Name: keywords, dtype: object
96    [국민, 아파트, 의원, 미래, 서울, 뉴시스, 주택, 주거]
Name: keywords, dtype: object
148    [국민, 의원, 주택, 환상, 대한, 아파트, 대변인]
Name: keywords, dtype: object
150    [아파트, 주택, 임대, 의원, 단장, 서울, 환상]
Name: keywords, dtype: object
183    [아파트, 주택, 임대, 의원, 국민, 발언, 전세]
Name: keywords, dtype: object
174    [아파트, 강남, 대한, 주택, 환상]
Name: keywords, dtype: object
98    [의원, 국민, 아파트, 임대, 주택, 대한, 환상, 지적]
Name: keywords, dtype: object
84    [발언, 주택, 아파트, 임대, 대책, 국민, 비판, 전세]
Name: keywords, dtype: object
173    [주택, 임대, 의원, 아파트, 발언, 민주당]
Name: keywords, dtype: object
280    [국민, 주택, 임대, 아파트, 의원, 뉴스, 정책]
Name: keywords, dtype: object
0    [민주당, 아파트, 의원, 국민, 주택, 결과, 중도]
Name: keywords, dtype: object
147    [의원, 전세, 임대, 아파트, 주택, 미래, 민주당, 거추, 진단장]
Name: keywords, dtype: object
155    [의원, 광주, 주택, 아파트, 윤희숙, 임대, 대한, 발언, 환상]
Name: keywords, dtype: object
195    [아파트, 의원, 주택, 임대, 환상, 국민, 